In [1]:
# Purpose
# Gather data from the reddit API
# Get the submissions from r/relationships between a time frame and top posts and other important stuff
# Insert the data into a mongo db

In [2]:
# Import and options
import praw
import pandas as pd
import datetime
import pprint

In [3]:
# Authentication
r = praw.Reddit(client_id='Ph0uQKkdUfbK4A',
                     client_secret='zjMGwiUhQkXCc4o9jXjR9ZIAQLc', password='geddippa',
                     user_agent='Relationships Bot 0.1', username='ooohandahh')

Version 4.4.0 of praw is outdated. Version 4.5.1 was released Monday May 08, 2017.


In [4]:
# Set the subreddit I'll be looking at
relationships = r.subreddit('relationships')

In [5]:
# Function to convert date to readable format
def get_date(submission):
    time = submission.created
    return datetime.datetime.fromtimestamp(time)

# Function to print the top level comment
def get_top_comment(submission):
    for top_level_comment in submission.comments:
        return(top_level_comment.body)

In [6]:
# Example of what I'll be pulling
for submission in relationships.top(limit=1):
    print("Title: ", submission.title)
    print("Flair:", submission.link_flair_text)
    print("Text: ", submission.selftext)
    print("Score: ", submission.score)
    print("Date:",get_date(submission))
    print("Top_Comment:", get_top_comment(submission))
    print("---------------------------------\n")

('Title: ', u'[UPDATE]My [26 F] with my husband [29 M] 1 year, he has been diagnosed with terminal cancer, how to make it count?')
('Flair:', u'Updates')
('Text: ', u'[Original Post](https://www.reddit.com/r/relationships/comments/3dheie/my_26_f_with_my_husband_29_m_1_year_he_has_been/)\n\nSummary of Original Post- My husband was diagnosed with terminal cancer. After failure of chemo and other treatments, he was told he has about a month to live. I asked for advice on how to make this time count and all of you wonderful people gave some very good advice.\n\n\nLet me begin by saying that as much as I was day dreaming that I\'ll get to update everyone of a miraculous recovery my brave darling made, I had to wake up to the bitter reality. My husband passed away a few days ago. And just as a lot of you had warned me, with all the preparation in the world and a warning that not many are fortunate to receive, absolutely nothing about when it actually happened felt like I was at all ready. It

In [7]:
import json
from pymongo import MongoClient

client = MongoClient()
db = client.relationships
posts = db.submissions

In [ ]:
db.collection_names()
posts.count()

In [ ]:
# Check if the whole month was inserted
for post in posts.find()[posts.count()-1:posts.count()]:
      pprint.pprint(post['date'])

In [ ]:
# Insert submissions into mongodb
for submission in relationships.submissions(1494806400, 1494979200):
#for submission in relationships.top(limit=1):
    data = {}
    data['title'] = submission.title
    data['flair'] = submission.link_flair_text
    data['text'] = submission.selftext
    data['score'] = submission.score
    data['date'] = get_date(submission)
    data['top_comment'] = get_top_comment(submission)
    posts.replace_one(data, data, upsert = True)
  
    

In [37]:
cursor = posts.aggregate([
    {"$group": {
        "_id":{"text":"$text", "top_comment":"$top_comment"},
        "title":{"$push":"$title"},
        "flair":{"$push":"$flair"},
        "date":{"$push":"$date"},
        "score":{"$push":"$score"},
        "first":{"$first":1}
    }}
], 
    allowDiskUse=True
)

In [38]:
posts2 = db.submissions2

In [39]:
posts2.count()

0

In [40]:
cursor.next()['title']

[u'Me [23 F] with my ex-gf/ great friend [25F] duration (6 months of dating/10 friendship)']

In [41]:
posts_dict = {}

for c in cursor:
    posts_dict = {}
    posts_dict['question'] = c['_id']['text']
    posts_dict['top_comment'] = c['_id']['top_comment']
    posts_dict['date'] = c['date']
    posts_dict['flair'] = c['flair']
    posts_dict['score'] = c['score']
    posts_dict['title'] = c['title']
    posts2.insert_one(posts_dict)

In [42]:
posts2.count()

33623